In [149]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mutual_info_score
from sklearn.linear_model import LinearRegression, RidgeCV,Ridge
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
import wget

In [2]:
url = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv'
wget.download(url,'car_data.csv')

'car_data.csv'

In [103]:
df = pd.read_csv('car_data.csv')
df

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11909,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,46120
11910,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,56670
11911,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,50620
11912,Acura,ZDX,2013,premium unleaded (recommended),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,50920


In [104]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11914 entries, 0 to 11913
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Make               11914 non-null  object 
 1   Model              11914 non-null  object 
 2   Year               11914 non-null  int64  
 3   Engine Fuel Type   11911 non-null  object 
 4   Engine HP          11845 non-null  float64
 5   Engine Cylinders   11884 non-null  float64
 6   Transmission Type  11914 non-null  object 
 7   Driven_Wheels      11914 non-null  object 
 8   Number of Doors    11908 non-null  float64
 9   Market Category    8172 non-null   object 
 10  Vehicle Size       11914 non-null  object 
 11  Vehicle Style      11914 non-null  object 
 12  highway MPG        11914 non-null  int64  
 13  city mpg           11914 non-null  int64  
 14  Popularity         11914 non-null  int64  
 15  MSRP               11914 non-null  int64  
dtypes: float64(3), int64(5

In [105]:
cols_used = ['make',
'model',
'year',
'engine_hp',
'engine_cylinders',
'transmission_type',
'vehicle_style',
'highway_mpg',
'city_mpg','price']

In [106]:
df.columns = df.columns.str.replace(' ', '_').str.lower()

In [107]:
df.rename(columns={'msrp':'price'},inplace=True)

In [108]:
df[cols_used] = df[cols_used].fillna(0)

In [166]:
df_sub = df[cols_used].copy()
df_sub

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,price
0,BMW,1 Series M,2011,335.0,6.0,MANUAL,Coupe,26,19,46135
1,BMW,1 Series,2011,300.0,6.0,MANUAL,Convertible,28,19,40650
2,BMW,1 Series,2011,300.0,6.0,MANUAL,Coupe,28,20,36350
3,BMW,1 Series,2011,230.0,6.0,MANUAL,Coupe,28,18,29450
4,BMW,1 Series,2011,230.0,6.0,MANUAL,Convertible,28,18,34500
...,...,...,...,...,...,...,...,...,...,...
11909,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,46120
11910,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,56670
11911,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,50620
11912,Acura,ZDX,2013,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,50920


In [167]:
#Q1
df_sub['transmission_type'].value_counts() #Automatic is the most frequent

AUTOMATIC           8266
MANUAL              2935
AUTOMATED_MANUAL     626
DIRECT_DRIVE          68
UNKNOWN               19
Name: transmission_type, dtype: int64

In [168]:
#Q2
df_sub.corr() # City_mpg & highway mpg has the highest correlation

C:\Users\user\AppData\Local\Temp\ipykernel_30644\518614899.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_sub.corr() # City_mpg & highway mpg has the highest correlation


,year,engine_hp,engine_cylinders,highway_mpg,city_mpg,price
year,1.000000,0.338714,-0.040708,0.258240,0.198171,0.227590
engine_hp,0.338714,1.000000,0.774851,-0.415707,-0.424918,0.650095
engine_cylinders,-0.040708,0.774851,1.000000,-0.614541,-0.587306,0.526274
highway_mpg,0.258240,-0.415707,-0.614541,1.000000,0.886829,-0.160043
city_mpg,0.198171,-0.424918,-0.587306,0.886829,1.000000,-0.157676
price,0.227590,0.650095,0.526274,-0.160043,-0.157676,1.000000


In [169]:
#Q3
price_avg = df_sub['price'].mean()
df_sub['above_average'] =np.where(df_sub['price']>price_avg,1,0)

In [170]:
df_sub['above_average'].value_counts()

0    8645
1    3269
Name: above_average, dtype: int64

In [171]:
df_sub.dtypes

make                  object
model                 object
year                   int64
engine_hp            float64
engine_cylinders     float64
transmission_type     object
vehicle_style         object
highway_mpg            int64
city_mpg               int64
price                  int64
above_average          int32
dtype: object

In [172]:
cat = df_sub.columns[list(df_sub.dtypes == 'object')]

In [173]:
X =  df_sub.drop(['price','above_average'],axis=1)
y = df_sub['above_average']
X[cat] = X[cat].astype('category')

X_train,X_test,y_train,y_test =train_test_split(X,y,test_size=0.2,random_state=42)
X_train,X_val,y_train,y_val =train_test_split(X_train,y_train,test_size=0.25,random_state=42)

# mutual_info_score(X_train,y_train)


In [174]:
#Q3
for col in X_train[cat].columns:
    print(col)
    print(round(mutual_info_score(X_train[col],y_train),2))
    print()

make
0.24

model
0.46

transmission_type
0.02

vehicle_style
0.08



In [175]:
#Q4

X =  df_sub.drop(['price','above_average'],axis=1)
X = pd.get_dummies(X,columns=cat)
y = df_sub['above_average']

X_train,X_test,y_train,y_test =train_test_split(X,y,test_size=0.2,random_state=42)
X_train,X_val,y_train,y_val =train_test_split(X_train,y_train,test_size=0.25,random_state=42)

model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(C=10, max_iter=1000, random_state=42, solver='liblinear')

In [176]:
y_pred = model.predict(X_test)
org_accuracy = accuracy_score(y_pred,y_test)
print(f"Accuracy: {org_accuracy:.3f}")

Accuracy: 0.933


In [182]:
#Q5 Feature elimination using Recurssive feature elimination
rfe = RFE(estimator=model)
rfe.fit(X_train, y_train)

feature_ranking = pd.DataFrame({'Feature': X_train.columns, 'Ranking': rfe.ranking_})
# feature_ranking.sort_values('Ranking', inplace=True)

for i, feature in enumerate(feature_ranking['Feature']):
    X_train_subset = X_train.drop(feature, axis=1)
    X_test_subset = X_test.drop(feature, axis=1)

    model.fit(X_train_subset, y_train)
    accuracy = model.score(X_test_subset, y_test)
    feature_ranking.at[i, 'accuracy_dif'] = abs(org_accuracy - accuracy)

In [183]:
feature_ranking

,Feature,Ranking,accuracy_dif
0,year,291,0.019303
1,engine_hp,219,0.024339
2,engine_cylinders,280,0.017625
3,highway_mpg,425,0.016786
4,city_mpg,186,0.016366
...,...,...,...
984,vehicle_style_Passenger Minivan,1,0.009652
985,vehicle_style_Passenger Van,1,0.016786
986,vehicle_style_Regular Cab Pickup,1,0.016786
987,vehicle_style_Sedan,106,0.011330


In [188]:
#Q6
X =  df_sub.drop(['price','above_average'],axis=1)
X = pd.get_dummies(X,columns=cat)
y = np.log1p(df_sub['price'])

X_train,X_test,y_train,y_test =train_test_split(X,y,test_size=0.2,random_state=42)
X_train,X_val,y_train,y_val =train_test_split(X_train,y_train,test_size=0.25,random_state=42)
a_list = [0, 0.01, 0.1, 1, 10]
RMSE = []
for a in a_list:
    ridge = Ridge(alpha=a,solver='sag',random_state=42)
    # Train the model on the training set
    ridge.fit(X_train, y_train)

    # Evaluate the model on the testing set
    y_pred_val = ridge.predict(X_val)
    y_pred_test = ridge.predict(X_test)
    # rmse1 = np.sqrt(mse(y_val,y_pred_val))
    rmse_val= mse(y_val, y_pred_val,squared=False)
    rmse_test = mse(y_test, y_pred_test,squared=False)
    RMSE.append(rmse_test)
    print(f'Alpha : {a}')
    print(f'RMSE_val : {round(rmse_val,3)}')
    print(f'RMSE_test : {round(rmse_test,3)}')
print(f'min RMSE : {round(min(RMSE),3)}')

C:\Users\user\miniconda3\envs\tensorflow_env\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Alpha : 0
RMSE_val : 0.487
RMSE_test : 0.486


C:\Users\user\miniconda3\envs\tensorflow_env\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Alpha : 0.01
RMSE_val : 0.487
RMSE_test : 0.486


C:\Users\user\miniconda3\envs\tensorflow_env\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Alpha : 0.1
RMSE_val : 0.487
RMSE_test : 0.486


C:\Users\user\miniconda3\envs\tensorflow_env\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Alpha : 1
RMSE_val : 0.487
RMSE_test : 0.486
Alpha : 10
RMSE_val : 0.487
RMSE_test : 0.487
min RMSE : 0.486


C:\Users\user\miniconda3\envs\tensorflow_env\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
